In [1]:
#import dependencies
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
from splinter import Browser 

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
#set the url and visit it by sprinter
url = 'https://developer.mapquest.com/documentation/tools/latitude-longitude-finder/'
browser.visit(url)

In [4]:
# read csv file as dataframe
nyc_bookstores_df = pd.read_csv('nyc_bookstores.csv')
nyc_bookstores_df

,Name,Address,Link
0,Berl's Brooklyn Poetry Shop,"141 Front St., Brooklyn, NY 11201",http://www.berlspoetry.com/
1,Bookmark Shoppe,"8415 3rd Ave, Brooklyn, NY 11209",https://www.bookmarkshoppe.com/
2,Books Are Magic,"225 Smith St., Brooklyn, NY 11231",https://www.booksaremagic.net/
3,Cafe Con Libros,"724 Prospect Pl. Brooklyn, NY 11216",https://www.cafeconlibrosbk.com/
4,Catland Books,"987 Flushing Ave, Brooklyn, NY 11206",https://www.catlandbooks.com/
...,...,...,...
63,Kew & Willow Books,"81-63 Lefferts Blvd, Kew Gardens\tNY, 11415",https://www.kewandwillow.com/
64,Topos Bookstore Cafe,"788 Woodward Ave., Ridgewood, NY 11385",https://www.toposbookstore.com/
65,Turn the Page Again!,"39-15 Bell Blvd, Bayside, NY 11361",http://www.turnthepageagain.com/
66,Everything Goes Book Cafe,"208 Bay St., Staten Island, NY 10301",http://www.etgstores.com/bookcafe/


In [ ]:
# nyc_bookstores_df.set_index('Name', inplace=True)

In [5]:
# Temp manual fixes to address column
nyc_bookstores_df['Address'][29] = '151 East Broadway, New York, NY 10002'
nyc_bookstores_df['Address'][59] = '238 W 10th St., New York, NY 10014'

In [7]:
# Split street address, state and zip
addresses = []
# cities = []
# states = []
zipcodes = []
for address in nyc_bookstores_df['Address']:
    split_address = address.split(',')
#     print(split_address)
    addresses.append(split_address[0])
    zipcodes.append(address[-5:])
nyc_bookstores_df['Street Address'] = addresses
nyc_bookstores_df['Zip Code'] = zipcodes

In [8]:
nyc_bookstores_df

,Name,Address,Link,Street Address,Zip Code
0,Berl's Brooklyn Poetry Shop,"141 Front St., Brooklyn, NY 11201",http://www.berlspoetry.com/,141 Front St.,11201
1,Bookmark Shoppe,"8415 3rd Ave, Brooklyn, NY 11209",https://www.bookmarkshoppe.com/,8415 3rd Ave,11209
2,Books Are Magic,"225 Smith St., Brooklyn, NY 11231",https://www.booksaremagic.net/,225 Smith St.,11231
3,Cafe Con Libros,"724 Prospect Pl. Brooklyn, NY 11216",https://www.cafeconlibrosbk.com/,724 Prospect Pl. Brooklyn,11216
4,Catland Books,"987 Flushing Ave, Brooklyn, NY 11206",https://www.catlandbooks.com/,987 Flushing Ave,11206
...,...,...,...,...,...
63,Kew & Willow Books,"81-63 Lefferts Blvd, Kew Gardens\tNY, 11415",https://www.kewandwillow.com/,81-63 Lefferts Blvd,11415
64,Topos Bookstore Cafe,"788 Woodward Ave., Ridgewood, NY 11385",https://www.toposbookstore.com/,788 Woodward Ave.,11385
65,Turn the Page Again!,"39-15 Bell Blvd, Bayside, NY 11361",http://www.turnthepageagain.com/,39-15 Bell Blvd,11361
66,Everything Goes Book Cafe,"208 Bay St., Staten Island, NY 10301",http://www.etgstores.com/bookcafe/,208 Bay St.,10301


In [9]:
# Get lat-lng results using address column
find_button = browser.find_by_id('btnGC')
clear_button = browser.find_by_id('clearDiv')
results = []
# for address in nyc_bookstores_df['Address']:
for index in range(68):
#     browser.find_by_name('street').type(address)
    browser.find_by_name('street').type(nyc_bookstores_df.loc[index,'Street Address'])
    browser.find_by_name('postalCode').type(nyc_bookstores_df.loc[index,'Zip Code'])
    find_button.click()
    result = browser.find_by_id('resultDiv').text
    results.append(result)
    time.sleep(1.5)
    clear_button.click()

In [10]:
# Clean up results and get latitudes and longtitudes seperately
latitudes= []
longitudes = []
for item in results:
    result_list = item.split(',')
    lat_result = result_list[0].split(':')
    lng_result = result_list[1].split(':')
    latitudes.append(float(lat_result[1]))
    longitudes.append(float(lng_result[1]))


In [ ]:
# browser.find_by_name('street').type('141 Front St')
# browser.find_by_name('postalCode').type('11201')
# find_button.click()

In [11]:
# Add latitude and Lontitude to the df
nyc_bookstores_df['Latitudes'] = latitudes
nyc_bookstores_df['Longitudes'] = longitudes
nyc_bookstores_df

,Name,Address,Link,Street Address,Zip Code,Latitudes,Longitudes
0,Berl's Brooklyn Poetry Shop,"141 Front St., Brooklyn, NY 11201",http://www.berlspoetry.com/,141 Front St.,11201,39.743943,-105.020089
1,Bookmark Shoppe,"8415 3rd Ave, Brooklyn, NY 11209",https://www.bookmarkshoppe.com/,8415 3rd Ave,11209,40.702443,-73.987360
2,Books Are Magic,"225 Smith St., Brooklyn, NY 11231",https://www.booksaremagic.net/,225 Smith St.,11231,40.624585,-74.030610
3,Cafe Con Libros,"724 Prospect Pl. Brooklyn, NY 11216",https://www.cafeconlibrosbk.com/,724 Prospect Pl. Brooklyn,11216,40.684065,-73.992261
4,Catland Books,"987 Flushing Ave, Brooklyn, NY 11206",https://www.catlandbooks.com/,987 Flushing Ave,11206,40.674445,-73.952519
...,...,...,...,...,...,...,...
63,Kew & Willow Books,"81-63 Lefferts Blvd, Kew Gardens\tNY, 11415",https://www.kewandwillow.com/,81-63 Lefferts Blvd,11415,40.747289,-73.942081
64,Topos Bookstore Cafe,"788 Woodward Ave., Ridgewood, NY 11385",https://www.toposbookstore.com/,788 Woodward Ave.,11385,40.708201,-73.830803
65,Turn the Page Again!,"39-15 Bell Blvd, Bayside, NY 11361",http://www.turnthepageagain.com/,39-15 Bell Blvd,11361,40.703704,-73.903679
66,Everything Goes Book Cafe,"208 Bay St., Staten Island, NY 10301",http://www.etgstores.com/bookcafe/,208 Bay St.,10301,40.764792,-73.771613


In [21]:
# Temp - manually fixing the seemingly random error on the first row
nyc_bookstores_df['Latitudes'][0] = 40.702443
nyc_bookstores_df['Longitudes'][0] = -73.98736

/Users/samin/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/samin/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# Save dataframe as csv
nyc_bookstores_df.to_csv('nyc_bookstores_location.csv',index=False)

In [25]:
# Close browser
browser.quit()